In [1]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision import transforms, models
import torch.nn.functional as F
import json
import requests
import pandas as pd

In [2]:
# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the transformation for the images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
# Download the ImageNet class index to label mapping
class_idx = json.loads(requests.get('https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json').text)
idx_to_class = {int(k): v[1] for k, v in class_idx.items()}

In [4]:
# Load a pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model = model.to(device)
model.eval()

C:\Users\richt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\richt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
def load_images(base_path, n_assemblies=None):
    all_images = {}
    assembly_folders = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    
    # Limit the number of assemblies if specified
    if n_assemblies is not None:
        assembly_folders = assembly_folders[:n_assemblies]

    for assembly_id in assembly_folders:
        assembly_path = os.path.join(base_path, assembly_id)
        all_images[assembly_id] = {
            'assembly': None,
            'bodies': {}
        }
    
        for image_name in os.listdir(assembly_path):
            if not image_name.endswith('.png'):
                continue
            image_path = os.path.join(assembly_path, image_name)
            with Image.open(image_path) as img:
                if image_name == 'assembly.png':
                    all_images[assembly_id]['assembly'] = np.array(img)
                else:
                    body_id = image_name.rstrip('.png')
                    all_images[assembly_id]['bodies'][body_id] = np.array(img)

    return all_images


In [6]:
def predict_image(image, model, transform, topk=5):
    # If image is a path, then load it; otherwise, assume it's already an array
    if isinstance(image, str):
        image = Image.open(image)
    else:
        image = Image.fromarray(image)
    
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)
    
    with torch.no_grad():
        output = model(image)
        probs, indices = output.topk(topk)
        probs = F.softmax(probs, dim=1)[0] * 100
        indices = indices[0]
    
    labels = [idx_to_class[int(idx.item())] for idx in indices]
    return labels, probs.cpu().numpy()

In [7]:
def predict_for_images(all_images, model, transform):
    predictions = {}
    
    for assembly_id, images in all_images.items():
        predictions[assembly_id] = {
            'assembly': None,
            'bodies': {}
        }
        
        # Predict for assembly image
        labels, probabilities = predict_image(images['assembly'], model, transform)
        predictions[assembly_id]['assembly'] = (labels, probabilities)
        
        # Predict for body images
        for body_id, body_image in images['bodies'].items():
            labels, probabilities = predict_image(body_image, model, transform)
            predictions[assembly_id]['bodies'][body_id] = (labels, probabilities)
            
    return predictions

In [8]:
def predictions_to_dataframe(all_predictions, topk=5):
    # Lists to store data for assemblies and bodies
    assembly_data = []
    body_data = []

    for assembly_id, predictions in all_predictions.items():
        # Process assembly predictions
        assembly_labels, assembly_probs = predictions['assembly']
        assembly_dict = {
            'assembly_id': assembly_id,
        }
        for i in range(topk):
            assembly_dict[f'assembly_prediction_{i + 1}'] = assembly_labels[i]
            assembly_dict[f'assembly_probability_{i + 1}'] = assembly_probs[i]
        assembly_data.append(assembly_dict)

        # Process body predictions
        for body_id, (body_labels, body_probs) in predictions['bodies'].items():
            body_dict = {
                'assembly_id': assembly_id,
                'body_id': body_id
            }
            for i in range(topk):
                body_dict[f'prediction_{i + 1}'] = body_labels[i]
                body_dict[f'probability_{i + 1}'] = body_probs[i]
            body_data.append(body_dict)

    # Convert lists to dataframes
    df_assembly_predictions = pd.DataFrame(assembly_data)
    df_body_predictions = pd.DataFrame(body_data)
    
    return df_assembly_predictions, df_body_predictions

In [9]:
# Load images
base_path = "C:\\Users\\richt\\Documents\\ASME_data\\train\\Fusion360GalleryDataset_23hackathon_train"
all_images = load_images(base_path, n_assemblies=50)

In [10]:
# Make predictions
all_predictions = predict_for_images(all_images, model, transform)

# [Further code for storing predictions and merging with df_main if needed]

In [11]:
# Convert predictions to dataframes
df_assembly_predictions, df_body_predictions = predictions_to_dataframe(all_predictions)

In [12]:
df_assembly_predictions.tail(50), df_body_predictions.tail()

(        assembly_id assembly_prediction_1  assembly_probability_1  \
 0   100029_94515530              joystick               25.794172   
 1   100106_7f144e5b             water_jug               32.202747   
 2   100112_bc0a563a             projector               53.157669   
 3   100126_e58fbfba               printer               44.617619   
 4   100138_119e1068              joystick               76.917374   
 5   100142_b2d7897a        potter's_wheel               71.566742   
 6   100221_4d7b66c4                 modem               47.319733   
 7   100236_568d1ccd              joystick               51.452179   
 8   100258_55065c89             hard_disc               50.729733   
 9   100261_362d7e17               spatula               93.719315   
 10  100261_66bf0666             hard_disc               31.994667   
 11  100326_0602c712               barbell               77.851089   
 12  100326_a57bf973               barbell               76.646477   
 13  100340_6b59be4d

In [13]:
# Loading the df_main.pickle file
df_main = pd.read_pickle("df_main.pkl")
df_main.shape

(92940, 33)

In [14]:
# Start with df_main
merged_df = df_main.copy()

# Merge with assembly predictions
merged_df = merged_df.merge(df_assembly_predictions, on='assembly_id', how='left')

# Merge with body predictions
merged_df = merged_df.merge(df_body_predictions, on=['assembly_id', 'body_id'], how='left', suffixes=('', '_body'))

# Handle any NaN values that might arise due to missing predictions
for col in merged_df.columns:
    if 'prediction' in col or 'probability' in col:
        merged_df[col].fillna('', inplace=True)

In [18]:
pd.set_option('display.max_columns', None)
merged_df.tail(10)

,body_id,assembly_id,name,volume,area,material_category,properties_vertex_count,properties_edge_count,properties_face_count,properties_loop_count,properties_shell_count,properties_body_count,properties_area,properties_volume,properties_likes_count,properties_comments_count,properties_views_count,properties_category_Mechanical Engineering,properties_category_Product Design,properties_category_Miscellaneous,properties_category_Electronics,properties_category_Machine design,properties_category_Furniture + Household,properties_category_Tools,properties_category_Automotive,properties_category_Design,properties_category_Toys,properties_industry_Other Industries,"properties_industry_Architecture, Engineering & Construction",properties_industry_Media & Entertainment,properties_industry_Civil Infrastructure,properties_industry_Product Design & Manufacturing,components_name,assembly_prediction_1,assembly_probability_1,assembly_prediction_2,assembly_probability_2,assembly_prediction_3,assembly_probability_3,assembly_prediction_4,assembly_probability_4,assembly_prediction_5,assembly_probability_5,prediction_1,probability_1,prediction_2,probability_2,prediction_3,probability_3,prediction_4,probability_4,prediction_5,probability_5
0,0b40d208-0529-11ec-b1f1-020dc2b44123,100221_4d7b66c4,NaN,1.183269,13.793986,Metal_Ferrous_Steel,1065,1686,690,759,20,20,2435.963990,422.013372,0,0,263,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,Hinge,modem,47.319733,hard_disc,35.732193,lighter,9.51869,notebook,3.901838,mousetrap,3.527543,broom,37.25832,ladle,31.347466,swab,11.604931,plunger,10.5338,pole,9.255492
1,0b41206c-0529-11ec-87d0-020dc2b44123,100221_4d7b66c4,NaN,188.236351,1194.122208,Plastic,1065,1686,690,759,20,20,2435.963990,422.013372,0,0,263,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,Base,modem,47.319733,hard_disc,35.732193,lighter,9.51869,notebook,3.901838,mousetrap,3.527543,tray,88.361671,scale,6.71702,maze,3.53478,modem,0.736748,book_jacket,0.649783
2,0b419630-0529-11ec-a6a6-020dc2b44123,100221_4d7b66c4,NaN,112.561776,737.410188,Plastic,1065,1686,690,759,20,20,2435.963990,422.013372,0,0,263,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,Lid,modem,47.319733,hard_disc,35.732193,lighter,9.51869,notebook,3.901838,mousetrap,3.527543,modem,42.605553,hard_disc,28.758694,notebook,16.462742,digital_clock,6.806357,mouse,5.366653
3,0b420ad4-0529-11ec-aac9-020dc2b44123,100221_4d7b66c4,NaN,1.646435,41.360040,Plastic,1065,1686,690,759,20,20,2435.963990,422.013372,0,0,263,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,NaN,modem,47.319733,hard_disc,35.732193,lighter,9.51869,notebook,3.901838,mousetrap,3.527543,screw,58.73246,corkscrew,14.461121,dumbbell,11.53425,hook,8.816345,letter_opener,6.455828
4,0b427fe4-0529-11ec-ba06-020dc2b44123,100221_4d7b66c4,NaN,0.094115,2.612833,Metal_Ferrous_Steel,1065,1686,690,759,20,20,2435.963990,422.013372,0,0,263,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,01-1785,modem,47.319733,hard_disc,35.732193,lighter,9.51869,notebook,3.901838,mousetrap,3.527543,iPod,90.626625,loudspeaker,4.26378,remote_control,1.83932,cassette,1.816347,hook,1.453929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,c741daee-0571-11ec-9491-06214ff114d7,102108_c62db547,NaN,169.164899,735.951376,Plastic,144,212,84,120,2,2,1471.902752,338.329798,0,0,273,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,,,,,,,,,,,,,,,,,,,,
496,cfa01ae4-0532-11ec-8c29-020a4e46e3ef,102177_dd672c16,NaN,44.328440,306.424026,Other,3539,6016,2596,2693,11,11,5660.631767,6717.360286,3,1,979,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,speaker,,,,,,,,,,,,,,,,,,,,
497,cfad885a-0532-11ec-94c8-020a4e46e3ef,102177_dd672c16,NaN,37.888788,263.484653,Other,3539,6016,2596,2693,11,11,5660.631767,6717.360286,3,1,979,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,blank,,,,,,,,,,,,,,,,,,,,
498,cfad885a-0532-11ec-94c8-020a4e46e3ef,102177_dd672c16,NaN,37.888788,263.484653,Other,3539,6016,2596,2693,11,11,5660.631767,6717.360286,3,1,979,0,0,0,0,0,0,0,0,

In [16]:
merged_df.to_pickle("df_main_images.pkl")